<a href="https://colab.research.google.com/github/RubenFranc/CDA_Taller2/blob/main/model/ProductsClasificator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.1 MB/s eta 0:00:00


In [5]:
#Numpy y Pandas
import pandas as pd
import numpy as np
from numpy import asarray

#Librerías de Scikit Learn
import sklearn
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

#Librerías de Keras y Tensor Flow
import tensorflow as tf
import keras_tuner as kt

from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Input, Rescaling, Lambda, MaxPooling2D, Conv2D, AveragePooling2D, Activation, Reshape
from keras import backend as K
from keras import applications
from keras.utils import plot_model
from tensorflow.keras.optimizers import Adam

#Gráficos
import matplotlib.pyplot as plt
%matplotlib inline

#Otras Librerías
import os
from PIL import Image
from tqdm import tqdm

#Verificamos las versiones de las librerías
print(f"La versión de sklearn es: {sklearn.__version__}")
print('Tensorflow version:', tf.__version__)

La versión de sklearn es: 1.5.2
Tensorflow version: 2.17.0


In [6]:
def convertImagesToArrays(dataset:str)->tuple:
    data='./data/'+dataset+'/Fruit'
    categories=os.listdir(data)
    im_size=50
    dataToArray=[]
    labels=[]
    for category in range(len(categories)):
        path=train_data+'/'+categories[category]
        label=category
        l=os.listdir(path)
        for im in tqdm(range(len(l))):
            new_im=Image.open(os.path.join(path,l[im])).resize((im_size,im_size))
            im_array=np.asanyarray(new_im)
            dataToArray.append(im_array)
            labels.append(label)

    num_images=len(dataToArray)
    index=np.int64(np.linspace(0,num_images-1,num_images))
    np.random.shuffle(index)
    s_data=[]
    s_labels=[]
    for i in tqdm(range(len(index))):
        try:
            s_data.append(dataToArray[index[i]])
            s_labels.append(labels[index[i]])
        except Exception as e:
            pass
    s_dataToArray=np.array(s_data)
    s_labels=np.array(s_labels)
    return s_dataToArray,s_labels

In [7]:
s_train_data,s_train_labels=convertImagesToArrays('train')
s_test_data,s_test_labels=convertImagesToArrays('test')
s_val_data,s_val_labels=convertImagesToArrays('val')

FileNotFoundError: [Errno 2] No such file or directory: './data/train/Fruit'

In [ ]:
s

ModuleNotFoundError: No module named 'tensorflow.keras'